<a href="https://colab.research.google.com/github/Seboldju/OCR-Stromzaehler-Arbeitsdokument/blob/main/Stromz%C3%A4hler_OCR_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation von OCR

In [ ]:
brew install tesseract


Bild aus Ordner einlesen

In [ ]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpeg"):
            img = cv2.imread(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
    return images

folder_path = 'path/to/your/folder'
images = load_images_from_folder(folder_path)


Bild in Graustufen umwandeln

In [ ]:
gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]


Bild entzerren

In [ ]:
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

rotated_images = [rotate_image(img, 270) for img in gray_images]  # Beispiel für 270 Grad Rotation


Erkennung der Kanten

In [ ]:
edges_images = [cv2.Canny(img, 50, 150) for img in rotated_images]


Finden von Konturen

In [ ]:
contours_list = []
for edges in edges_images:
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_list.append(contours)


Ziffern aus dem Bild ausschneiden

In [ ]:
digit_images_list = []
for i, contours in enumerate(contours_list):
    digit_images = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 15 and h > 30:  # Filter für Größe der Ziffern
            digit_images.append(rotated_images[i][y:y+h, x:x+w])
    digit_images_list.append(digit_images)


OCR und Ziffernerkennung

In [ ]:
import pytesseract

all_digits = []
for digit_images in digit_images_list:
    digits = []
    for digit_image in digit_images:
        digit = pytesseract.image_to_string(digit_image, config='--psm 10')
        digits.append(digit.strip())
    all_digits.append(digits)

for i, digits in enumerate(all_digits):
    print(f"Erkannte Ziffern für Bild {i}: {digits}")


Speicherung der Ergebnisse als Datei

In [ ]:
import csv

with open('results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Bild', 'Erkannte Ziffern'])
    for i, digits in enumerate(all_digits):
        writer.writerow([i, ''.join(digits)])
